# Import repo

In [1]:
!git clone https://github.com/FernandoDoreto/convert-streamlit-to-django.git

fatal: destination path 'convert-streamlit-to-django' already exists and is not an empty directory.


In [2]:
%cd /content/convert-streamlit-to-django

/content/convert-streamlit-to-django


In [3]:
pwd

'/content/convert-streamlit-to-django'

# Load dataset

In [4]:
pip install streamlit

In [18]:
from src.processing.data_management import LoadIrisDataset

In [19]:
df = LoadIrisDataset()
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


* Train Test split

In [20]:
# from app_pages.page_ml_ClfIrisSpecies import ML_ClfIrisSpeciesBody
from config import config
from src.processing.data_management import TrainTestSplit
# from src.machine_learning.train_pipeline import TrainPipeline_ClfIrisSpecies
# import src.machine_learning.pipeline as pipeline

In [21]:
 X_train, X_test,y_train, y_test = TrainTestSplit(df=df,TARGET=config.ClfIrisSpecies_TARGET)

# Grid Search on 1 pipeline

In [30]:
from config import config
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV

ClfIrisSpecies_DT = Pipeline(
    [       
        ("feat_selection",SelectFromModel(DecisionTreeClassifier(random_state=config.RANDOM_STATE))),
        ("feat_scaling",StandardScaler()),
        ("model", DecisionTreeClassifier(random_state=config.RANDOM_STATE))
    ]
)


_parameters = {
    'model__splitter': ["best","random"],
    'model__max_depth': [None,3,5,10],
    'model__criterion': ["gini", "entropy"]
}


_pipe = GridSearchCV(
		estimator = ClfIrisSpecies_DT,
		param_grid = _parameters, 
		cv=5,n_jobs=-2,verbose=2)

In [31]:
_pipe.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__

[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_dept

[Parallel(n_jobs=-2)]: Done  80 out of  80 | elapsed:    0.5s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('feat_selection',
                                        SelectFromModel(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                                         class_weight=None,
                                                                                         criterion='gini',
                                                                                         max_depth=None,
                                                                                         max_features=None,
                                                                                         max_leaf_nodes=None,
                                                                                         min_impurity_decrease=0.0,
                                                                                         min_impurity_s

In [27]:
_pipe.best_estimator_

Pipeline(memory=None,
         steps=[('feat_selection',
                 SelectFromModel(estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                  class_weight=None,
                                                                  criterion='gini',
                                                                  max_depth=None,
                                                                  max_features=None,
                                                                  max_leaf_nodes=None,
                                                                  min_impurity_decrease=0.0,
                                                                  min_impurity_split=None,
                                                                  min_samples_leaf=1,
                                                                  min_samples_split=2,
                                                                  min_weight_fract

# Grid Search on Stacking pipeline

In [32]:
from config import config
from sklearn.pipeline import Pipeline

from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [33]:
ClfIrisSpecies_DT = Pipeline(
    [       
        ("feat_selection",SelectFromModel(DecisionTreeClassifier(random_state=config.RANDOM_STATE))),
        ("feat_scaling",StandardScaler()),
        ("model", DecisionTreeClassifier(random_state=config.RANDOM_STATE))
    ]
)


_parameters_DT = {
    'model__splitter': ["best","random"],
    'model__max_depth': [None,3,5,10],
    'model__criterion': ["gini", "entropy"]
}


PipelineClfIrisSpecies_DT = GridSearchCV(
		estimator = ClfIrisSpecies_DT,
		param_grid = _parameters_DT, 
		cv = 5,n_jobs = -2,verbose = 2)

In [34]:
ClfIrisSpecies_RF = Pipeline(
    [       
        ("feat_selection",SelectFromModel(RandomForestClassifier(random_state=config.RANDOM_STATE))),
        ("feat_scaling",StandardScaler()),
        ("model", RandomForestClassifier(random_state=config.RANDOM_STATE))
    ]
)

_parameters_RF = {
    'model__n_estimators': [100,50,200],
    'model__max_depth': [None,3,5,10],
    'model__criterion': ["gini", "entropy"]
}

PipelineClfIrisSpecies_RF = GridSearchCV(
		estimator = ClfIrisSpecies_RF,
		param_grid = _parameters_RF, 
		cv = 5,n_jobs = -2,verbose = 2)

In [35]:
ClfIrisSpecies_GB = Pipeline(
    [       
        ("feat_selection",SelectFromModel(GradientBoostingClassifier(random_state=config.RANDOM_STATE))),
        ("feat_scaling",StandardScaler()),
        ("model", GradientBoostingClassifier(random_state=config.RANDOM_STATE,learning_rate=0.1))
    ]
)

_parameters_GB = {
    'model__learning_rate': [1e-1,1e-2,5e-1],
    'model__n_estimators': [100,800]
}


PipelineClfIrisSpecies_GB = GridSearchCV(
		estimator = ClfIrisSpecies_GB,
		param_grid = _parameters_GB, 
		cv = 5,n_jobs = -2,verbose = 2)

In [36]:
ClfIrisStacking = StackingClassifier(
    classifiers=[
        PipelineClfIrisSpecies_GB,
        PipelineClfIrisSpecies_RF,
        PipelineClfIrisSpecies_DT
    ],
    use_probas=True,
    meta_classifier=LogisticRegression(solver='liblinear')
)


In [37]:
ClfIrisStacking.fit(X_train,y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............


[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............


[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=100 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=100, total=   0.3s
[CV] model__learning_rate=0.1, model__n_estimators=800 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=800, total=   1.2s
[CV] model__learning_rate=0.1, model__n_estimators=800 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=800, total=   1.2s
[CV] model__learning_rate=0.1, model__n_estimators=800 ...............
[CV]  model__learning_rate=0.1, model__n_estimators=800, total=   1.2s
[CV] model__learning_rate=0.1, model__n_estimators=800 ...............
[CV]  

...

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=None, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=None, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=None, model__

[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=best 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=best, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_depth=10, model__splitter=random, total=   0.0s
[CV] model__criterion=gini, model__max_depth=10, model__splitter=random 
[CV]  model__criterion=gini, model__max_dept

[Parallel(n_jobs=-2)]: Done  80 out of  80 | elapsed:    0.5s finished


StackingClassifier(average_probas=False,
                   classifiers=[GridSearchCV(cv=5, error_score=nan,
                                             estimator=Pipeline(memory=None,
                                                                steps=[('feat_selection',
                                                                        SelectFromModel(estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                                                                                             criterion='friedman_mse',
                                                                                                                             init=None,
                                                                                                                             learning_rate=0.1,
                                                                                                                             loss='deviance',
           

In [ ]:
ClfIrisStacking.predict_proba(X_test)

In [ ]:
for i,clf in enumerate(ClfIrisStacking.classifiers):
  print(ClfIrisStacking.classifiers[i].best_estimator_)
  print("\n\n ----------------------------------------------------------------------------- \n\n")

In [59]:
model_pipe = ClfIrisStacking.classifiers[0]
model_pipe.best_estimator_

AttributeError: ignored